# Programación Entera: Heurísticas

## Descripción

[TODO]

## Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
## Fecha

  * Mayo de 2018


## Contenidos
  
  * [Problema de Transporte](#Problema-de-Transporte)
  * [Problema de Asignación](#Problema-de-Asignación)
    * [Asignación Clásico](#Asignación-Clásico)
    * [Asignación Generalizado](#Asignación-Generalizado)
  * [Problema de la Mochila](#Problema-de-la-Mochila)
    * [Mochila Simple](#Mochila-Simple)
    * [Mochila Múltiple](#Mochila-Múltiple)
    * [Mochila Multidimensional](#Mochila-Multidimensional)

In [1]:
library(lpSolve)
library(magrittr)

In [2]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

## Funciones de Apoyo

In [3]:
dataset.folder <- "./data"

In [4]:
ReadDataSet <- function() {
    
}

In [5]:
Solve <- function(direction = "max", f.obj, f.con, f.rhs, f.dir = rep("<=", nrow(f.con)), ...) {
    lp(direction, f.obj, f.con, f.dir, f.rhs, ...)
}

## Problema de Transporte

#### Descripción:
TODO

#### Modelo:
TODO

In [6]:
TransportationExact <- function(demand, capacity, cost, direction = "min", ...) {
    m <- length(capacity)
    n <- length(demand)    
    
    conditions <- matrix(0, nrow = n + m, ncol = n * m)
    for (i in 1:m) {
        conditions[i, seq(n) + (i - 1) * n] <- 1
    }
    for (i in (m+1):(n+m)) {
        conditions[i, seq(i - m, m * n, by = n)] <- 1
    }
    res <- c(rep("<=", m), rep(">=", n))
    return(Solve(direction, cost, conditions, c(capacity, demand), res, ...))
}

#### Heurísticas:
TODO

In [7]:
TransportationGreedy <- function(demand, capacity, cost) {
    m <- length(capacity)
    n <- length(demand) 
    sol <- list()
    
    sol$solution <- matrix(0, nrow = m, ncol = n)
    sol$objval <- 0
    while (!all(demand == 0)) {
        ij <- which(cost == min(cost[capacity > 0, demand > 0]), arr.ind = TRUE)
        ij.slice <- which(capacity[ij[, 1]] > 0 & demand[ij[, 2]] > 0)[1]
        i <- ij[ij.slice, 1]
        j <- ij[ij.slice, 2]
        inc <- min(capacity[i], demand[j])    
        
        sol$solution[i, j] <- sol$solution[i, j] + inc
        capacity[i] <- capacity[i] - inc
        demand[j] <- demand[j] - inc
        sol$objval <- sol$objval + cost[i, j] * inc
    }
    return(sol)
}

### Ejemplos

In [8]:
transportation.problem.1 <- list(
    demand = c(45, 20, 30, 30),
    capacity = c(35, 60, 48),
    cost = matrix(c(8, 6, 10, 9,
                    9, 12,13, 7,
                    14,9, 16, 5), nrow = 3, byrow = TRUE)
)

In [9]:
a <- TransportationExact(transportation.problem.1$demand,
                    transportation.problem.1$capacity,
                    transportation.problem.1$cost)

matrix(a$solution, 
       nrow = 3, byrow =TRUE)
print(a$objval)

35,0,0,0
10,20,12,0
0,0,18,30


[1] 976


In [10]:
a <- TransportationGreedy(transportation.problem.1$demand,
                    transportation.problem.1$capacity,
                    transportation.problem.1$cost)

matrix(a$solution, 
       nrow = dim(transportation.problem.1$cost)[1], 
       ncol = dim(transportation.problem.1$cost)[2])
print(a$objval)

15,20,0,0
30,0,30,0
0,0,0,30


[1] 1050


## Problema de Asignación

### Asignación Clásico

#### Descripción:
TODO

#### Modelo:
TODO

In [11]:
AssignmentExact <- function(cost, ...) {
    return(TransportationExact(rep(1, dim(cost)[1]), rep(1, dim(cost)[2]), cost, ...))
}

#### Heurísticas:
TODO

In [12]:
AssignmentGreedy <- function(cost) {    
    return(TransportationGreedy(rep(1, dim(cost)[1]), rep(1, dim(cost)[2]), cost))
}

In [13]:
GreedyLocalSearchImprovement <- function(change.1, change.2, cost) {
    return(cost[change.1[, drop = FALSE]] + 
           cost[change.2[, drop = FALSE]] -
           cost[change.1[1], change.2[2]] -
           cost[change.2[1], change.1[2]])
}

In [14]:
GreedyLocalSearchBest <- function(cost, changes) {
    best = list(v = 0, i = 0, j = 0)
    for (i in 1:nrow(changes)) {
        for(j in 1:nrow(changes)) {
            v.temp <- GreedyLocalSearchImprovement(changes[i, ,drop = FALSE], 
                                                   changes[j, , drop = FALSE],
                                                   cost)
            if (best$v < v.temp) {
                best$v <- v.temp
                best$i <- i
                best$j <- j
            }
        }
    }
    return(best)
}

In [15]:
AssignmentGreedyLocalSearch <- function(cost) {    
    sol <- AssignmentGreedy(cost)
        
    changes <- which(sol$solution != 0, arr.ind = TRUE)
    best = GreedyLocalSearchBest(cost, changes)
    while(best$v > 0) {
        best = best = GreedyLocalSearchBest(cost, changes)

        sol$solution[changes[best$i, , drop = FALSE]] <- 0
        sol$solution[changes[best$j, , drop = FALSE]] <- 0
        
        temp <- changes[best$i, 2]
        changes[best$i, 2] <- changes[best$j, 2]
        changes[best$j, 2] <- temp
        
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 1
        sol$solution[changes[best$j, , drop = FALSE]] <- 1
        
        sol$objval <- sol$objval - best$v
    }
    return(sol)
}

### Ejemplos

In [16]:
(matriz.15x15.1 <- as.matrix(read.csv("./data/assignment/matriz_15x15_1.csv", header = FALSE)))

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
58,66,6,77,7,19,44,22,9,90,86,11,61,11,26
56,78,12,30,89,20,62,14,82,76,70,100,59,57,42
93,80,90,77,71,96,55,25,46,57,16,30,91,55,0
85,48,55,36,91,10,31,98,42,84,98,91,76,93,10
62,85,32,56,21,56,78,48,49,66,0,39,39,6,53
44,64,25,47,56,63,80,35,22,70,100,96,32,84,70
50,23,8,90,23,51,98,94,33,84,95,41,77,33,30
9,32,100,36,3,21,66,19,91,48,81,57,86,96,35
59,62,12,98,93,38,59,93,18,21,59,87,76,9,82
37,1,13,29,85,18,29,20,54,94,30,1,0,67,51


In [17]:
a <- AssignmentExact(matriz.15x15.1)
matrix(a$solution, nrow = dim(matriz.15x15.1)[1])
a$objval

0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


[1] 135

In [18]:
AssignmentGreedy(matriz.15x15.1)

0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [19]:
AssignmentGreedyLocalSearch(matriz.15x15.1)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [20]:
(matriz.15x15.2 <- as.matrix(read.csv("./data/assignment/matriz_15x15_2.csv", header = FALSE)))

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
6,51,28,33,12,43,24,72,15,93,12,0,83,28,43
36,29,45,40,61,88,6,70,73,64,12,86,35,42,13
77,40,77,49,54,71,85,8,99,16,96,61,74,16,66
79,48,43,25,84,96,48,74,65,21,49,23,9,83,7
100,23,36,2,44,28,66,15,80,45,70,47,89,18,33
5,50,92,82,96,41,75,92,86,12,15,74,41,89,25
62,5,81,41,84,23,69,87,37,24,29,31,55,80,81
64,46,60,98,40,57,76,23,17,84,99,6,65,27,65
84,0,28,16,81,85,24,21,88,5,76,47,22,23,33
3,74,78,80,97,10,86,30,16,74,88,35,96,56,99


In [21]:
a <- AssignmentExact(matriz.15x15.2)
matrix(a$solution, nrow = dim(matriz.15x15.2)[1])
a$objval

0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


[1] 125

In [22]:
AssignmentGreedy(matriz.15x15.2)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
AssignmentGreedyLocalSearch(matriz.15x15.2)

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
(matriz.15x15.3 <- as.matrix(read.csv("./data/assignment/matriz_15x15_3.csv", header = FALSE)))

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
60,40,71,35,28,40,51,79,61,43,76,12,43,0,96
93,8,72,6,44,88,76,36,83,16,12,9,84,77,31
28,24,61,5,49,12,18,74,96,82,85,63,48,80,45
13,33,54,73,92,86,47,66,6,99,60,2,77,16,57
15,48,83,80,23,8,66,75,41,64,12,49,36,28,15
92,15,89,42,28,41,84,100,7,40,70,50,43,81,96
43,6,25,3,23,65,70,89,25,69,76,24,30,21,29
24,40,56,65,98,86,45,74,33,5,78,29,17,46,85
27,84,0,5,83,22,96,86,25,55,99,85,40,16,21
33,6,99,30,47,23,74,65,64,32,80,23,39,23,52


In [25]:
a <- AssignmentExact(matriz.15x15.3)
matrix(a$solution, nrow = dim(matriz.15x15.3)[1])
a$objval

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


[1] 153

In [26]:
AssignmentGreedy(matriz.15x15.3)

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [27]:
AssignmentGreedyLocalSearch(matriz.15x15.3)

0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### Asignación Generalizado

In [28]:
ReadGeneralizedAssignment <- function(file.name) {
    con <- 
        file.name %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit(" ") %>%
        unlist() %>%
        as.integer()
    close(con)
    list(
        m = raw[1],
        n = raw[2],
        cost = matrix(raw[3:(2 + raw[1] * raw[2])], 
                      nrow = raw[1], ncol = raw[2], byrow = TRUE),
        resource = matrix(raw[(3 + raw[1] * raw[2]):(2 + 2 * raw[1] * raw[2])], 
                          nrow = raw[1], ncol = raw[2], byrow = TRUE),
        capacity = raw[(3 + 2 * raw[1] * raw[2]):length(raw)]
        
    ) %>%
    return()
}

#### Descripción:
TODO

#### Modelo:
TODO

In [29]:
GeneralizedAssignmentExact <- function(problem.statement, direction = "max", ...) {    
    
    m <- problem.statement$m
    n <- problem.statement$n
    cost <- problem.statement$cost
    resource <- problem.statement$resource
    capacity <- problem.statement$capacity
    demand <- rep(1, n)
    
    
    conditions <- matrix(0, nrow = n + m, ncol = n * m)
    for (i in 1:m) {
        conditions[i, seq(n) + (i - 1) * n] <- resource[i, ]
    }
    for (i in (m + 1):(n + m)) {
        conditions[i, seq(i - m, m * n, by = n)] <- 1
    }
    res <- c(rep("<=", m), rep("=", n))
    return(Solve(direction, t(cost), conditions, c(capacity, demand), res, all.bin = TRUE, ...))
}

#### Heurísticas:
TODO

In [30]:
GeneralizedAssignmentGreedy <- function(cost) {    
    
}

### Ejemplos

In [31]:
(gap2.1 <- ReadGeneralizedAssignment("./data/assignment/gap2_1.txt"))

15,12,16,15,20,18,11,14,23,14,18,13,11,15,17,18,20,11,20,24
22,14,10,15,12,21,24,16,17,22,11,15,18,15,18,23,22,23,21,13
25,11,22,10,22,17,22,12,21,21,16,14,23,21,14,17,25,18,15,15
23,13,14,25,21,18,23,19,17,18,20,20,22,24,23,13,17,22,20,21
15,25,22,14,17,21,16,21,20,12,15,18,17,21,20,20,13,14,23,14
11,11,5,25,10,17,16,20,5,20,14,10,15,6,11,14,13,8,15,11
17,17,12,23,16,25,13,16,19,16,24,17,24,6,6,16,25,21,5,18
13,13,20,17,8,7,16,9,23,18,21,24,11,15,12,24,19,23,13,12
6,10,18,16,12,25,17,7,10,6,20,20,9,20,11,21,6,9,20,23
7,14,18,24,7,24,18,19,18,23,15,16,14,9,16,25,15,20,23,17


In [32]:
a <- GeneralizedAssignmentExact(gap2.1)
matrix(a$solution, nrow = gap2.1$m, byrow = TRUE)
a$objval

0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0


[1] 434

In [33]:
(gap4.1 <- ReadGeneralizedAssignment("./data/assignment/gap4_1.txt"))

15,16,12,17,12,15,19,15,19,17,25,17,19,15,20,16,12,16,13,22,16,16,14,21,21,24,13,25,21,22
12,20,11,13,22,15,14,22,21,10,24,22,23,14,14,22,22,17,14,19,21,11,20,21,14,23,10,17,24,22
19,19,16,11,18,20,18,22,21,15,22,23,16,23,18,18,18,22,21,12,20,18,13,25,10,23,21,24,22,18
23,12,13,12,24,23,11,20,14,20,19,14,19,22,24,13,10,16,12,23,18,24,24,18,25,10,18,18,19,15
15,13,16,23,19,21,15,12,18,21,12,18,21,16,17,14,13,14,19,18,21,12,14,17,15,24,21,13,11,24
7,13,21,19,10,23,14,16,13,16,17,11,23,20,9,24,24,19,18,21,18,18,18,17,24,19,9,9,7,13
12,12,23,23,6,24,13,24,16,17,9,15,5,20,19,17,16,13,8,13,24,5,6,25,10,16,23,8,6,13
11,17,17,21,12,13,16,17,15,11,16,20,23,18,14,24,23,21,6,20,13,10,20,15,20,10,8,23,24,18
18,23,13,8,15,12,16,13,9,6,12,13,13,14,18,6,6,7,12,10,22,10,8,8,9,18,5,7,21,22
22,7,7,13,12,15,13,12,23,16,21,9,15,16,6,18,11,16,22,6,9,7,15,9,6,16,17,23,5,10


In [34]:
a <- GeneralizedAssignmentExact(gap4.1)
matrix(a$solution, nrow = gap4.1$m, byrow = TRUE)
a$objval

0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0
0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


[1] 656

In [35]:
(gap6.1 <- ReadGeneralizedAssignment("./data/assignment/gap6_1.txt"))

21,22,17,16,17,24,17,15,25,18,18,21,19,23,24,25,25,16,22,20,20,20,20,21,24,19,22,15,20,17,17,22
18,24,18,22,19,25,23,15,24,22,19,19,20,25,23,25,20,16,22,24,23,17,24,18,18,22,15,20,17,19,15,22
24,20,20,15,21,15,21,22,19,16,16,19,21,21,15,17,24,17,19,18,21,25,17,24,21,24,25,17,19,24,21,22
21,20,16,21,20,18,22,17,18,19,16,15,21,24,23,19,20,20,16,16,21,17,17,20,22,21,16,16,15,17,24,24
20,19,20,25,17,22,17,18,20,23,20,16,17,24,18,16,20,23,22,15,15,23,18,21,25,20,18,19,24,25,17,16
17,24,22,23,17,20,19,16,24,21,25,16,25,23,22,23,15,15,15,20,20,21,23,24,21,25,23,22,25,24,21,17
15,22,23,24,17,23,16,16,18,19,15,15,23,18,24,25,18,25,18,15,16,22,19,18,19,22,18,23,16,15,25,25
20,18,18,24,25,20,16,19,25,24,25,24,15,22,21,23,15,24,17,15,24,16,25,19,19,22,17,16,15,20,20,18
12,24,13,15,19,7,6,25,15,19,12,24,24,25,15,19,15,24,8,18,9,23,10,8,11,10,18,21,20,14,20,16
10,20,10,24,10,25,11,11,23,14,6,16,5,20,15,11,11,19,5,7,25,9,5,24,6,13,13,25,19,18,22,7
24,6,16,19,18,9,20,23,19,23,15,10,25,25,12,17,16,8,10,9,9,21,19,11,14,22,20,23,23,9,8,25


In [36]:
a <- GeneralizedAssignmentExact(gap6.1)
matrix(a$solution, nrow = gap6.1$m, byrow = TRUE)
a$objval

0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0
0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


[1] 761

In [37]:
(gap8.1 <- ReadGeneralizedAssignment("./data/assignment/gap8_1.txt"))

21,25,18,18,21,25,22,18,18,23,17,24,16,21,24,20,23,24,23,24,19,24,21,20,25,24,22,23,18,25,20,17,17,20,17,17,23,19,18,15,19,24,24,25,16,23,17,18
19,16,22,23,24,21,18,18,17,17,25,21,24,17,17,22,15,24,18,21,21,23,19,24,19,22,17,18,24,24,21,24,16,21,23,22,19,20,24,20,19,21,24,25,22,15,19,16
23,19,16,20,20,18,24,15,24,17,24,21,15,19,22,20,20,24,20,16,17,19,18,18,17,24,23,15,16,22,25,24,22,16,25,20,16,20,18,15,24,24,15,24,23,24,18,25
16,16,21,22,20,22,22,19,25,16,18,19,22,19,18,22,25,20,23,17,18,21,16,20,16,15,21,20,18,16,23,16,22,22,18,22,17,15,15,18,15,20,15,21,19,23,17,25
16,15,22,23,16,23,20,20,23,16,19,16,18,20,18,22,16,15,22,24,24,21,22,16,19,22,17,20,20,21,15,15,21,16,16,15,15,16,24,25,19,24,22,22,20,19,17,24
24,19,21,21,25,24,20,25,21,24,18,21,21,15,23,24,18,23,20,25,22,20,17,25,16,22,19,18,22,25,16,16,22,23,22,23,16,21,17,21,20,16,15,23,24,24,18,22
24,23,18,22,25,18,16,21,24,25,18,16,16,23,25,15,16,19,18,20,19,18,24,18,22,16,25,25,22,18,16,17,15,25,24,23,23,21,25,22,24,19,16,21,16,15,16,25
15,21,23,24,17,23,23,20,18,17,16,25,16,22,20,17,21,18,17,17,18,24,24,24,24,18,20,21,23,24,19,18,18,19,22,16,23,16,17,22,18,21,24,25,25,22,18,19
7,6,8,6,13,16,9,14,15,5,10,6,10,7,11,10,13,7,11,11,9,15,10,11,10,14,16,7,12,12,7,11,15,16,11,8,13,10,5,7,6,9,6,5,11,9,13,10
10,7,5,10,16,14,15,10,16,15,16,16,5,12,16,6,12,12,10,9,15,11,13,8,12,10,13,14,12,13,9,7,8,14,6,14,15,12,10,9,7,7,9,16,6,5,13,13
16,8,13,16,16,11,5,14,10,5,12,13,12,15,16,5,16,5,15,6,7,9,13,13,11,9,14,13,8,12,5,13,8,9,14,7,13,14,8,9,14,14,12,10,10,5,14,8


In [38]:
# a <- GeneralizedAssignmentExact(gap8.1)
# matrix(a$solution, nrow = gap8.1$m, byrow = TRUE)
# a$objval

In [39]:
(gap12.1 <- ReadGeneralizedAssignment("./data/assignment/gap12_1.txt"))

25,18,15,16,18,25,19,18,25,23,19,25,19,21,24,16,18,23,18,17,25,20,24,21,17,19,24,25,17,19,19,25,20,17,21,20,20,17,19,25,25,19,23,24,20,16,21,24,17,15,17,19,15,21,25,16,21,20,24,22
21,21,15,23,23,22,20,22,21,24,20,19,23,24,23,23,22,22,15,18,20,19,20,24,22,20,20,20,20,17,20,24,18,19,22,17,21,17,19,17,22,22,18,22,20,17,19,15,15,19,23,22,24,23,17,21,25,23,20,19
21,17,24,17,24,15,22,24,20,18,21,23,19,24,23,22,22,21,16,18,21,22,25,22,17,25,22,22,24,22,25,20,16,19,23,20,18,23,23,19,19,21,17,22,15,16,23,24,17,22,23,18,24,16,20,25,19,17,22,21
21,20,22,15,23,24,25,22,18,16,23,24,20,21,15,21,17,20,24,22,24,25,16,17,18,20,25,18,24,24,24,22,19,20,15,20,17,19,16,23,15,22,22,24,15,16,20,17,21,24,24,24,24,25,19,24,25,16,22,19
24,23,22,19,18,16,23,18,22,24,25,16,23,21,17,18,19,17,25,17,18,23,24,18,24,23,23,15,19,18,24,19,19,24,24,24,15,21,21,24,24,19,23,24,21,23,24,22,18,19,24,15,17,17,25,17,24,16,19,21
16,21,19,22,16,16,17,23,25,16,18,22,15,25,20,15,18,23,22,16,17,17,16,24,24,16,17,23,19,23,15,17,24,19,24,24,24,16,22,16,18,23,23,20,16,25,23,17,18,23,24,19,22,19,19,18,21,21,22,17
23,23,19,18,19,19,21,24,22,16,17,25,24,17,23,17,20,25,25,22,17,24,17,17,23,23,16,17,19,19,23,24,17,25,18,19,18,16,17,17,24,23,17,19,15,23,22,17,23,21,15,19,18,22,23,16,16,22,16,21
24,16,15,25,16,15,20,20,22,18,21,19,21,19,17,17,17,22,25,17,20,25,15,17,20,16,23,20,23,22,23,15,21,24,21,23,19,24,17,24,25,18,24,21,20,18,22,15,16,18,23,24,25,20,20,21,17,15,24,21
18,18,18,19,18,24,22,19,25,15,16,23,17,16,18,22,21,18,21,18,20,18,16,24,17,22,24,18,22,18,24,23,23,25,20,23,18,22,21,18,24,17,15,25,23,16,22,24,18,21,16,24,22,18,25,23,23,18,16,22
16,18,19,18,18,24,24,23,25,16,20,22,25,16,20,18,19,24,24,25,22,23,23,24,18,24,16,21,17,22,22,16,15,17,19,23,15,19,22,22,23,23,16,20,22,24,20,20,23,16,24,17,22,17,19,25,16,16,19,16
17,25,6,14,16,12,19,19,5,16,23,16,15,22,19,10,25,7,24,9,7,24,22,23,10,25,21,16,13,17,9,6,16,23,11,22,14,22,5,14,9,19,24,16,6,21,19,11,22,22,19,24,12,20,15,17,21,17,25,19


In [40]:
# a <- GeneralizedAssignmentExact(gap12.1)
# matrix(a$solution, nrow = gap12.1$m, byrow = TRUE)
# a$objval

## Problema de la Mochila

#### Descripción:
TODO

### Mochila Simple

#### Descripción:
TODO

#### Modelo:
TODO

In [41]:
KnapsackSimpleExact <- function(object, capacity) {
    
}

#### Heurísticas:
TODO

In [42]:
KnapsackSimpleGreedy <- function(object, capacity) {
    
}

### Mochila Múltiple

#### Descripción:
TODO

#### Modelo:
TODO

In [43]:
KnapsackMultipleExact <- function(object, capacity) {
    
}

#### Heurísticas:
TODO

In [44]:
KnapsackMultipleGreedy <- function(object, capacity) {
    
}

### Mochila Multidimensional

#### Descripción:
TODO

#### Modelo:
TODO

In [45]:
KnapsackMultidimensionalExact <- function(object, capacity, ...) {
    n <- length(object$value)
    m <- length(capacity)    
    return(Solve("max", object$value, t(object$weight), capacity, all.bin = TRUE, ...))
}

#### Heurísticas:
TODO

In [46]:
KnapsackMultidimensionalGreedy <- function(object, capacity) {
    
}

### Ejemplos

In [47]:
(knapsack.multidimensional.problem.1 <- list(
    object = list(
        value = c(12, 13, 15, 10),
        weight = matrix(c(0.5, 0.3, 0.2,
                          1.0, 0.8, 0.2,
                          1.5, 1.5, 0.1,
                          3.1, 2.5, 0.4), nrow = 4, byrow = TRUE) 
    ),
    capacity = c(3.1, 2.5, 0.4)
))

0.5,0.3,0.2
1.0,0.8,0.2
1.5,1.5,0.1
3.1,2.5,0.4


In [48]:
a <- KnapsackMultidimensionalExact(
    knapsack.multidimensional.problem.1$object, 
    knapsack.multidimensional.problem.1$capacity
)
matrix(a$solution, nrow = 1, byrow = TRUE)
a$objval

0,1,1,0


[1] 28